<a href="https://colab.research.google.com/github/Sheega-ameenah/Anomaly_detection_with_CLIP/blob/main/ClipAnomaly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Checking pre-installed pytorch versions

In [ ]:
!pip install torch torchvision
import torch
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")

Torch version: 2.5.1+cu121
CUDA available: True
CUDA device: Tesla T4


In [ ]:
!pip install transformers


In [ ]:
from transformers import CLIPProcessor, CLIPModel
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
print("Hugging Face CLIP loaded!")

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

Hugging Face CLIP loaded!


Download the MVTec AD  dataset


In [ ]:
!wget -q https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420938113-1629952094/mvtec_anomaly_detection.tar.xz
!tar -xf mvtec_anomaly_detection.tar.xz

xz: (stdin): Unexpected end of input
tar: Unexpected EOF in archive
tar: Unexpected EOF in archive
tar: Error is not recoverable: exiting now


In [ ]:
!xz -t mvtec_anomaly_detection.tar.xz


xz: mvtec_anomaly_detection.tar.xz: No such file or directory


In [ ]:
!rm -f mvtec_anomaly_detection.tar.xz


In [ ]:
!pwd
!ls -lh


/content
total 9.1G
dr-xr-x--- 5 2133  111 4.0K Mar  4  2019 bottle
dr-xr-x--- 5 2133  111 4.0K Mar  4  2019 cable
dr-xr-x--- 5 2133  111 4.0K Mar  4  2019 capsule
dr-xr-x--- 5 2133  111 4.0K Mar  4  2019 carpet
dr-xr-x--- 5 2133  111 4.0K Mar  4  2019 grid
dr-xr-x--- 5 2133  111 4.0K Mar  4  2019 hazelnut
dr-xr-x--- 5 2133  111 4.0K Mar  4  2019 leather
-r--r----- 1 2133  111  21K Mar  7  2019 license.txt
-rw-r--r-- 1 root root 4.2G Jan 16 21:24 mvtec_anomaly_detection.tar.xz.1
-rw-r--r-- 1 root root 5.0G Aug 26  2021 mvtec_anomaly_detection.tar.xz.2
-r-xr----- 1 2133  111  881 Mar  7  2019 readme.txt
drwxr-xr-x 1 root root 4.0K Jan 15 14:25 sample_data


In [ ]:
!rm mvtec_anomaly_detection.tar.xz.1


In [ ]:
!xz -t mvtec_anomaly_detection.tar.xz.2


In [ ]:
!mv mvtec_anomaly_detection.tar.xz.2 mvtec_anomaly_detection.tar.xz


In [ ]:
!tar -xf mvtec_anomaly_detection.tar.xz


In [ ]:
!ls -l


total 5141684
dr-xr-x--- 5 2133  111       4096 Mar  4  2019 bottle
dr-xr-x--- 5 2133  111       4096 Mar  4  2019 cable
dr-xr-x--- 5 2133  111       4096 Mar  4  2019 capsule
dr-xr-x--- 5 2133  111       4096 Mar  4  2019 carpet
dr-xr-x--- 5 2133  111       4096 Mar  4  2019 grid
dr-xr-x--- 5 2133  111       4096 Mar  4  2019 hazelnut
dr-xr-x--- 5 2133  111       4096 Mar  4  2019 leather
-r--r----- 1 2133  111      20845 Mar  7  2019 license.txt
dr-xr-x--- 5 2133  111       4096 Mar  4  2019 metal_nut
-rw-r--r-- 1 root root 5264982680 Aug 26  2021 mvtec_anomaly_detection.tar.xz
dr-xr-x--- 5 2133  111       4096 Mar  6  2019 pill
-r-xr----- 1 2133  111        881 Mar  7  2019 readme.txt
drwxr-xr-x 1 root root       4096 Jan 15 14:25 sample_data
dr-xr-x--- 5 2133  111       4096 Mar  4  2019 screw
dr-xr-x--- 5 2133  111       4096 Mar  4  2019 tile
dr-xr-x--- 5 2133  111       4096 Mar  4  2019 toothbrush
dr-xr-x--- 5 2133  111       4096 Mar  4  2019 transistor
dr-xr-x--- 5 2133  111 

Checking necessary libraries


In [ ]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.2 MB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-2bd3hh4h
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-2bd3hh4h
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369489 sha256=6052d4563964252c0d12f011f2ea210252ffdb8e5f90335f8c6e60055d385281
  Stored in directory: /tmp/pip-ephem-wheel-cache-fx6sgorl/wheels/3f/7c/a4/9b490845988bf7a4db33674d52f709f088f64392063872eb9a
Successfully built clip


In [ ]:
import os
import clip
from torchvision import transforms
from PIL import Image


Loading the CLIP model

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)


100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 129MiB/s]


In [20]:
dataset_path = "/content"
categories = os.listdir(dataset_path)
print(f"Categories: {categories}")


Categories: ['.config', 'screw', 'license.txt', 'cable', 'leather', 'toothbrush', 'carpet', 'readme.txt', 'bottle', 'wood', 'transistor', 'grid', 'capsule', 'metal_nut', 'hazelnut', 'tile', 'zipper', 'pill', 'mvtec_anomaly_detection.tar.xz', 'sample_data']


prep preprocessing functions

In [21]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


Implement One-Shot Anomaly Detection

In [22]:
def compute_similarity(reference_image_path, test_image_path):
    # Load and preprocess images
    reference_image = preprocess(Image.open(reference_image_path)).unsqueeze(0).to(device)
    test_image = preprocess(Image.open(test_image_path)).unsqueeze(0).to(device)

    # Extract CLIP features
    with torch.no_grad():
        reference_features = model.encode_image(reference_image)
        test_features = model.encode_image(test_image)

    # Normalize features
    reference_features /= reference_features.norm(dim=-1, keepdim=True)
    test_features /= test_features.norm(dim=-1, keepdim=True)

    # Compute cosine similarity
    similarity = (reference_features @ test_features.T).item()
    return similarity


Testing the Anomaly Detection

In [23]:
category = "bottle"  # Example: replace with other categories
normal_image_path = os.path.join(dataset_path, category, "train", "good", "000.png")  # Example path
test_image_path = os.path.join(dataset_path, category, "test", "broken_large", "001.png")  # Example path

similarity = compute_similarity(normal_image_path, test_image_path)
print(f"Similarity between normal and test image: {similarity:.4f}")

# Threshold for anomaly detection
threshold = 0.8
if similarity < threshold:
    print("Anomaly Detected!")
else:
    print("No Anomaly Detected.")


Similarity between normal and test image: 0.9087
No Anomaly Detected.


Batch Evaluation

In [25]:
def evaluate_category(category, threshold=0.8):
    normal_image_path = os.path.join(dataset_path, category, "train", "good", "000.png")  # Single reference image
    test_dir = os.path.join(dataset_path, category, "test")

    results = []
    for defect_type in os.listdir(test_dir):
        defect_path = os.path.join(test_dir, defect_type)
        for test_image in os.listdir(defect_path):
            test_image_path = os.path.join(defect_path, test_image)
            similarity = compute_similarity(normal_image_path, test_image_path)
            is_anomaly = similarity < threshold
            results.append((test_image, similarity, is_anomaly))

    return results


In [26]:
results = evaluate_category("bottle", threshold=0.8)
for image_name, similarity, is_anomaly in results:
    print(f"{image_name} - Similarity: {similarity:.4f} - {'Anomaly' if is_anomaly else 'Normal'}")


012.png - Similarity: 0.9653 - Normal
016.png - Similarity: 0.9756 - Normal
001.png - Similarity: 0.9692 - Normal
008.png - Similarity: 0.9595 - Normal
007.png - Similarity: 0.9619 - Normal
010.png - Similarity: 0.9761 - Normal
017.png - Similarity: 0.9653 - Normal
015.png - Similarity: 0.9727 - Normal
004.png - Similarity: 0.9756 - Normal
002.png - Similarity: 0.9736 - Normal
005.png - Similarity: 0.9668 - Normal
014.png - Similarity: 0.9668 - Normal
018.png - Similarity: 0.9712 - Normal
009.png - Similarity: 0.9766 - Normal
003.png - Similarity: 0.9722 - Normal
006.png - Similarity: 0.9502 - Normal
011.png - Similarity: 0.9731 - Normal
000.png - Similarity: 0.9624 - Normal
013.png - Similarity: 0.9746 - Normal
019.png - Similarity: 0.9731 - Normal
012.png - Similarity: 0.9487 - Normal
016.png - Similarity: 0.9570 - Normal
001.png - Similarity: 0.9531 - Normal
008.png - Similarity: 0.9590 - Normal
007.png - Similarity: 0.9795 - Normal
010.png - Similarity: 0.9390 - Normal
017.png - Si